<a href="https://colab.research.google.com/github/2303A52183/2303a52183-GEN-AI-2025/blob/main/GEN_AI_ASS_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import load_model

def root_mean_absolute_error(y_true, y_pred):
    # Cast y_true to have the same type as y_pred (float32) to avoid the error
    y_true = tf.cast(y_true, y_pred.dtype)
    return tf.sqrt(tf.reduce_mean(tf.abs(y_true - y_pred)))

file_path = '/content/Housing (3).csv'
data = pd.read_csv(file_path)


X = data.drop('price', axis=1)
y = data['price']

categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=[np.number]).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

model = Sequential()
model.add(Dense(18, activation='swish', input_dim=X_train.shape[1]))
model.add(Dense(26, activation='swish'))
model.add(Dense(20, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=RMSprop(), metrics=[root_mean_absolute_error])

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

model.save('housing_price_model.h5')

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Training RMSE: {train_rmse}")
print(f"Testing RMSE: {test_rmse}")

loaded_model = load_model('housing_price_model.h5', custom_objects={"root_mean_absolute_error": root_mean_absolute_error})
def predict_housing_price(features):
    # The 'prefarea' column is missing in the user_input
    # So, we adjusted the columns to match the user_input
    features_df = pd.DataFrame([features], columns=['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'furnishingstatus'])

    # Add the 'prefarea' column and set a default value
    features_df['prefarea'] = 'no'  # or any other appropriate default value

    features = pipeline.transform(features_df)
    prediction = loaded_model.predict(features)
    return prediction[0][0]

user_input = [8000, 3, 2, 2, 1, 'yes', 'no', 'no', 'yes', 'no', 'furnished']
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price}")


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 25525774974976.0000 - root_mean_absolute_error: 2170.1577 - val_loss: 30129988304896.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 25186988457984.0000 - root_mean_absolute_error: 2167.3181 - val_loss: 30129986207744.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 26721977892864.0000 - root_mean_absolute_error: 2194.0593 - val_loss: 30129977819136.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 25218206662656.0000 - root_mean_absolute_error: 2170.6877 - val_loss: 30129969430528.0000 - val_root_mean_absolute_error: 2258.2996
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 23960492179456.0000 - root_mean_absolute_error: 2143.1345 - val_loss: 30129954750464.0000 - val_root_mean_absolute_error: 2258.2993
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2468

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


Training RMSE: 3809314.821346222
Testing RMSE: 4254441.3790842155
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Predicted Housing Price: 1575885.625
